[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/NNHieu/TrueAnswer_ZaloAIChallenge2019_VietnameseWikiQA/blob/master/PlayWithData.ipynb)

## Import lib và set dir

In [1]:
from sklearn.feature_extraction.text import TfidfVectorizer,CountVectorizer
import pandas as pd
import os
import json

### Nếu chạy bằng Colab nhớ điền đường dãn đến thư mục Dataset ở drive

In [2]:
if 'google.colab' in str(get_ipython()):
    print("Đang chạy bằng google colab")
    from google.colab import drive
    drive.mount("/content/drive", force_remount=True)
    import os
    os.chdir("drive/My Drive/<đường dãn đến thư mục Dataset ở drive>")
else:
    print("Đang không chạy bằng google colab")
    os.chdir("./Dataset")

Đang không chạy bằng google colab


## Mở các file train.json và test.json

### Sơ qua dữ liệu dùng cho training:
Tên file: train.json

Bao gồm ~18K item đã được gán nhãn. Mỗi item bao gồm:

* **id**: id của mỗi item
* **question**: câu hỏi
* **text**: đoạn text `liên quan` đến câu hỏi, kết quả từ 1 dummy search tool. Là 1 đoạn Wikipedia article.
* **title**: title của Wikipedia article mà đoạn text được lấy từ.
* **label**: nhãn, 2 loại, `True` hoặc `False`, đoạn text trên `có trả lời cho câu hỏi` hoặc `không trả lời được`.

Như ở dưới thì câu hỏi `Quang Hải giành được chức vô địch U21 quốc gia năm bao nhiêu tuổi` có thể xuất hiện nhiều lần, nhưng đi kèm với đoạn text và label tương ứng khác nhau.

In [3]:
# Read Training Data
train_df = pd.read_json('./train.json', orient='records')
print('Training data shape: ',train_df.shape)
train_df.head(5)

Training data shape:  (18108, 5)


,id,question,title,text,label
0,u7-1570446247_1,Quang Hải giành được chức vô địch U21 quốc gia...,Nguyễn Quang Hải (sinh 1997),"Năm 2013 , Nguyễn Quang Hải giành chức vô địch...",True
1,u7-1570446247_2,Quang Hải giành được chức vô địch U21 quốc gia...,Nguyễn Quang Hải (sinh 1997),"Sau chức vô địch U-21 quốc gia 2013 , Nguyễn Q...",True
2,u7-1570446247_0,Quang Hải giành được chức vô địch U21 quốc gia...,Nguyễn Quang Hải (sinh 1997),Anh bắt đầu gia nhập lò đào tạo trẻ Hà Nội T&T...,False
3,u7-1570446247_3,Quang Hải giành được chức vô địch U21 quốc gia...,Nguyễn Quang Hải (sinh 1997),"Năm 2011 , Nguyễn Quang Hải mới 14 tuổi được g...",False
4,u7-1570445661_0,Mỗi hiệp bóng đá kéo dài bao lâu,Bóng đá,Một trận đấu bóng đá thông thường có hai hiệp ...,True


### Sơ qua dữ liệu dùng cho testing:
Tên file: test.json

Bao gồm các test cases có trong tập public test. Mỗi test case bao gồm:

* **__id__**: test case ID
* **question**: câu hỏi
* **paragraphs**: **các** đoạn văn có được từ 1 dummy search tool. Mỗi đoạn có 1 ID riêng **trong test case** (nghĩa là các đoạn trong các test case khác nhau có thể có cùng id). Các đội tham gia phải tìm ID của các đoạn có thể trả lời cho `câu hỏi`.
* **title**: title của Wikipedia article mà đoạn text được lấy từ.

In [4]:
# Read Test Data
test_df = pd.read_json('./test.json', orient='records')
print('Test data shape: ',test_df.shape)
test_df.head(5)

Test data shape:  (501, 4)


,__id__,question,title,paragraphs
0,test_0001,Toà nhà nào cao nhất Việt Nam,The Landmark 81,"[{'id': 'p1', 'text': 'The Landmark 81 là một ..."
1,test_0002,Messi thuận chân nào,Lionel Messi,"[{'id': 'p1', 'text': 'Đôi chân ngắn nhưng mạn..."
2,test_0003,Sao Hoả còn được gọi là gì,Sao Hỏa,"[{'id': 'p1', 'text': 'Sao Hoả còn gọi là : Ho..."
3,test_0004,Hồ Hoàn Kiếm nằm ở đâu,Hồ Hoàn Kiếm,"[{'id': 'p1', 'text': 'Hồ Hoàn Kiếm ( chữ Nôm ..."
4,test_0005,Lionel Messi sinh ra ở đâu,Lionel Messi,"[{'id': 'p1', 'text': 'Lionel Andrés "" Leo "" M..."


Ví dụ với câu hỏi trong testcase 2, `Messi thuận chân nào`?, có các đoạn paragraphs sau:

In [5]:
df =pd.DataFrame(test_df['paragraphs'][1])
df.head()

,id,text
0,p1,Đôi chân ngắn nhưng mạnh mẽ giúp Messi dễ dàng...
1,p2,"Cựu huấn luyện viên Barcelona , Pep Guardiola ..."
2,p3,Ngoài ra anh còn một chân sút phạt chính và ha...
3,p4,"Khi đề cập đến kĩ năng dắt bóng của anh , Mara..."
4,p5,"Ngày 11 tháng 3 năm 2010 , Messi được chọn làm..."


Để cho tiện, chuyển test set về cùng dạng với training set

In [14]:
with open('./test.json', 'r') as test_file:
    test_df = pd.json_normalize(json.load(test_file), 'paragraphs', meta=['__id__', 'question', 'title'], record_prefix='para_')
print("Số test case tính theo số đoạn text là: ", test_df.shape[0])
test_df.head(5)

Số test case tính theo số đoạn text là:  2678


,para_id,para_text,__id__,question,title
0,p1,The Landmark 81 là một toà nhà chọc trời trong...,test_0001,Toà nhà nào cao nhất Việt Nam,The Landmark 81
1,p2,"Toà tháp cao 81 tầng , hiện tại là toà nhà cao...",test_0001,Toà nhà nào cao nhất Việt Nam,The Landmark 81
2,p1,Đôi chân ngắn nhưng mạnh mẽ giúp Messi dễ dàng...,test_0002,Messi thuận chân nào,Lionel Messi
3,p2,"Cựu huấn luyện viên Barcelona , Pep Guardiola ...",test_0002,Messi thuận chân nào,Lionel Messi
4,p3,Ngoài ra anh còn một chân sút phạt chính và ha...,test_0002,Messi thuận chân nào,Lionel Messi


## Tìm hiều sâu hơn
### 1. Basic Data Exploration
#### 1.1 Thông tin cơ bản

In [7]:
# Null values and Data types
print('------------- Train Set -------------')
print(train_df.info())
print('------------- Test Set -------------')
print(test_df.info())

------------- Train Set -------------
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 18108 entries, 0 to 18107
Data columns (total 5 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   id        18108 non-null  object
 1   question  18108 non-null  object
 2   title     18108 non-null  object
 3   text      18108 non-null  object
 4   label     18108 non-null  bool  
dtypes: bool(1), object(4)
memory usage: 583.7+ KB
None
------------- Test Set -------------
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2678 entries, 0 to 2677
Data columns (total 5 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   para_id    2678 non-null   object
 1   para_text  2678 non-null   object
 2   __id__     2678 non-null   object
 3   question   2678 non-null   object
 4   title      2678 non-null   object
dtypes: object(5)
memory usage: 104.7+ KB
None


Có bị chỗ nào thiếu dữ liệu không ?

In [8]:
train_df.isnull().sum()

id          0
question    0
title       0
text        0
label       0
dtype: int64

In [9]:
test_df.isnull().sum()

para_id      0
para_text    0
__id__       0
question     0
title        0
dtype: int64

Không!!

In [150]:
print("Tỷ lệ Label trong trainning set")
train_df.groupby(['label']).size().reset_index().rename(columns={0: "count"})

Tỷ lệ Label trong trainning set


,label,count
0,False,12370
1,True,5738


#### 1.2 Các câu hỏi

In [86]:
print("Có tổng cộng", train_df['question'].value_counts().shape[0], "câu hỏi trong training set")

Có tổng cộng 5070 câu hỏi trong training set


In [87]:
print("Có tổng cộng", test_df['question'].value_counts().shape[0], "câu hỏi trong test set")

Có tổng cộng 501 câu hỏi trong test set


In [90]:
train_questions = set(train_df['question'].unique())
test_questions = set(test_df['question'].unique())
print("Các câu hỏi xuất hiện ở cả training set và test set")
train_questions.intersection(test_questions)


Các câu hỏi xuất hiện ở cả training set và test set


{'Bác Hồ ra đi tìm đường cứu nước năm bao nhiêu',
 'Kinh tế Việt Nam hiện nay đứng hàng thứ mấy trên thế giới',
 'Trụ sở chính của Liên Hiệp Quốc đặt ở đâu',
 'Vịnh Hạ Long nằm ở khu vực nào'}

In [108]:
print("Trong training set")
tmp_df = train_df.groupby(['question', 'title']).size()
# tmp_df['q_len'] = train_df.question.map(lambda x: len(x.split(" ")))
tmp2 = tmp_df.groupby(['question']).size().reset_index().rename(columns={0:'count'})
print("Số cặp (question, title):", tmp_df.shape[0])
print("Một câu hỏi có tối đa", tmp2.max()['count'], "title, và là câu")
train_df.loc[train_df['question'] == 'Quê hương của tổng thống Obama ở đâu'].head(5)

Trong training set
Số cặp (question, title): 15048
Một câu hỏi có tối đa 15 title, và là câu


,question,count,q_len
0,""" Dư địa chí "" là tác phẩm của ai",3,10
1,"""Bỉ vỏ"" là tác phẩm của nhà văn nào",2,9
2,"""Chiếu dời đô"" của vua Lý Công Uẩn ra đời vào ...",1,13
3,"""Chùa vàng"" là tên gọi khác của quốc gia nào",4,10
4,"""Cỗ xe tăng"" là tên gọi của đội bóng nào",4,10


In [139]:
tmp2['q_len'] = tmp2.question.map(lambda x: len(x.split(" ")))
print("Độ dài lớn nhất của câu hỏi là", tmp2['q_len'].max(), "từ, ngắn nhất", tmp2['q_len'].min(), "từ")
pd.concat([tmp2.nlargest(2, 'q_len'), tmp2.nsmallest(3, 'q_len')])

Độ dài lớn nhất của câu hỏi là 32 từ, ngắn nhất 3 từ


,question,count,q_len
2699,Phóng viên chiến trường nào của hãng thông tấn...,2,32
1810,"Lê Bích Phương, vận động viên duy nhất giành đ...",3,32
365,Bitcoin là gì,1,3
321,BDSM là gì,1,3
1253,Gfriend là gì,1,3


In [141]:
print("Trong test set")
tmp_df = test_df.groupby(['question', 'title']).size()
# tmp_df['q_len'] = train_df.question.map(lambda x: len(x.split(" ")))
tmp2 = tmp_df.groupby(['question']).size().reset_index().rename(columns={0:'count'})
print("Số cặp (question, title):", tmp_df.shape[0])
print("Một câu hỏi có tối đa", tmp2.max()['count'], "title")
tmp2['q_len'] = tmp2.question.map(lambda x: len(x.split(" ")))
print("Độ dài lớn nhất của câu hỏi là", tmp2['q_len'].max(), "từ, ngắn nhất", tmp2['q_len'].min(), "từ")
pd.concat([tmp2.nlargest(2, 'q_len'), tmp2.nsmallest(3, 'q_len')])

Trong test set
Số cặp (question, title): 501
Một câu hỏi có tối đa 1 title
Độ dài lớn nhất của câu hỏi là 21 từ, ngắn nhất 4 từ


,question,count,q_len
136,Hòn đảo là nơi duy nhất mà chủ tịch Hồ Chí Min...,1,21
232,Mỹ dỡ bỏ lệnh dỡ bỏ hoàn toàn cấm vận vũ khí á...,1,21
146,Khi nào phạt đền,1,4
172,Lionel Messi là ai,1,4
206,Messi thuận chân nào,1,4
